## Create SparkSession and import

In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import Window

sc = SparkContext('local')
spark = SparkSession(sc)

24/01/23 21:28:32 WARN Utils: Your hostname, krxps resolves to a loopback address: 127.0.1.1; using 192.168.68.61 instead (on interface wlp0s20f3)
24/01/23 21:28:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/23 21:28:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Create a dataframe (with SparkSession)

In [5]:
df = spark.createDataFrame([(1,), (2,)], "id: int")
df.show()

df.createOrReplaceTempView("test")

df_table = spark.table("test")
df_table.show()

df_sql = spark.sql("select id from test where id = 2")
df_sql.show()

df_range = spark.range(0, 3)
df_range.show()

+---+
| id|
+---+
|  1|
|  2|
+---+

+---+
| id|
+---+
|  1|
|  2|
+---+

+---+
| id|
+---+
|  2|
+---+

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



## Filter

In [12]:
df = spark.createDataFrame([(1, "a"), (2, "b"), (None, None)], ["id", "name"])

df.filter((col("id") == 1) | (col("name") == "a")).take(5)

df.filter("id == 1 or name == 'a'").take(5)

assert df.filter(col("id").isNull()).count() == 1
assert df.filter(isnull(col("id"))).count() == 1

assert df.filter(col("id").isNotNull()).count() == 2
assert df.filter(~isnull("id")).count() == 2
assert df.filter("id is not null").count() == 2

## Access column

In [14]:
df = spark.createDataFrame([(1,), (2,)], "id long")

print(df.id)
print(df["id"])
print(col("id"))

Column<'id[type]'>
Column<'id[type]'>
Column<'id'>


## Create column

In [18]:
df = spark.createDataFrame([(1,), (2,)], "id long")

df.withColumn("squared", pow("id", 2)).show()

+---+-------+
| id|squared|
+---+-------+
|  1|    1.0|
|  2|    4.0|
+---+-------+

root
 |-- id: long (nullable = true)



## Rename a column

In [8]:
df = spark.createDataFrame([(1, "test"), (2, "yo")], "id long, name string")

(df.withColumnRenamed("id", "id_test")
 .withColumn("name_test", col("name"))
 .show()
)

+-------+----+---------+
|id_test|name|name_test|
+-------+----+---------+
|      1|test|     test|
|      2|  yo|       yo|
+-------+----+---------+



## Literal

In [24]:
df = spark.range(0, 3)

(df
 .withColumn("squared", pow("id", lit(2)))
 .withColumn("function", lit("square"))
 .show()
)

+---+-------+--------+
| id|squared|function|
+---+-------+--------+
|  0|    0.0|  square|
|  1|    1.0|  square|
|  2|    4.0|  square|
+---+-------+--------+



## Select

In [13]:
df = spark.createDataFrame([(10, "a"), (20, "b"), (30, "c"), (40, "d"), (40, "e")], ["id", "name"])

print(df.select("id", "name").first())
df.select((col("id").between(20, 40) & col("name").isin("b", "c", "d")).alias("boolean")).show()


Row(id=10, name='a')
+-------+
|boolean|
+-------+
|  false|
|   true|
|   true|
|   true|
|  false|
+-------+



## Arrays

In [25]:
df = spark.createDataFrame([(1,)])

(df
 .withColumn("test", lit("a_b"))
 .withColumn("array", split(col("test"), "_"))
 .withColumn("second_element", col("array")[1])
 .withColumn("sorted", sort_array("array", asc=False))
 .withColumn("size", size("array"))
 .withColumn("contains", array_contains("array", "c"))
 .show()
)

+---+----+------+--------------+------+----+--------+
| _1|test| array|second_element|sorted|size|contains|
+---+----+------+--------------+------+----+--------+
|  1| a_b|[a, b]|             b|[b, a]|   2|   false|
+---+----+------+--------------+------+----+--------+



In [27]:
(df.withColumn("test", lit("a_b"))
 .withColumn("array", split("test", "_"))
 .withColumn("yo", explode("array"))
 .show()
)

+---+----+------+---+
| _1|test| array| yo|
+---+----+------+---+
|  1| a_b|[a, b]|  a|
|  1| a_b|[a, b]|  b|
+---+----+------+---+



## Manpulating Strings

In [31]:
df = spark.createDataFrame([('a',)], 'name string')

df.withColumn("upper", upper('name')).show()

+----+-----+
|name|upper|
+----+-----+
|   a|    A|
+----+-----+



In [34]:
(df.withColumn("description", lit("Description: bla bla"))
 .withColumn("result", regexp_replace("description", "^Description: ", ""))
 .show()
)

+----+--------------------+-------+
|name|         description| result|
+----+--------------------+-------+
|   a|Description: bla bla|bla bla|
+----+--------------------+-------+



## Remove duplicates

In [39]:
df = spark.createDataFrame([('a',), ('b',)], 'name string')

df_with_duplicates = df.withColumn("test", lit("yo"))
df_with_duplicates.show()

df_with_duplicates.dropDuplicates(["test"]).show()

df_with_duplicates.select("test").distinct().show()

+----+----+
|name|test|
+----+----+
|   a|  yo|
|   b|  yo|
+----+----+

+----+----+
|name|test|
+----+----+
|   a|  yo|
+----+----+

+----+
|test|
+----+
|  yo|
+----+



## Aggregations

In [43]:
df = spark.range(10)

df.agg(mean("id").alias("mean")).show()

+----+
|mean|
+----+
| 4.5|
+----+



In [42]:
df.agg(approx_count_distinct(col("id"), 0.15).alias("distincCount")).show()

+------------+
|distincCount|
+------------+
|          11|
+------------+



In [28]:
df = spark.createDataFrame([("A", 3), ("B", 30), ("B", 15), ("A", 100)], ["id", "value"])

df.groupBy("id").agg(max("value").alias("highest"), min("value").alias("lowest")).show()

+---+-------+------+
| id|highest|lowest|
+---+-------+------+
|  B|     30|    15|
|  A|    100|     3|
+---+-------+------+



In [47]:
df1 = spark.createDataFrame([
    Row(course="dotNET", year=2012, earnings=10000),
    Row(course="Java", year=2012, earnings=20000),
    Row(course="dotNET", year=2012, earnings=5000),
    Row(course="dotNET", year=2013, earnings=48000),
    Row(course="Java", year=2013, earnings=30000),
    Row(course="Kotlin", year=2013, earnings=1000),
    Row(course="Javascript", year=2013, earnings=100),
])
df1.groupBy("year").pivot("course", ["dotNET", "Java", "Kotlin"]).sum("earnings").show()


+----+------+-----+------+
|year|dotNET| Java|Kotlin|
+----+------+-----+------+
|2012| 15000|20000|  null|
|2013| 48000|30000|  1000|
+----+------+-----+------+



## Window

In [2]:
d = [
    {'name': 'Alice', 'age': 40, 'country': 'France', 'date': "2023-10-01"},
    {'name': 'Jane', 'age': 28, 'country': 'France', 'date': "2023-10-01"},
    {'name': 'Bob', 'age': 30, 'country': 'France', 'date': "2023-10-10"},
    {'name': 'Richard', 'age': 50, 'country': 'Allemagne', 'date': "2023-10-10"},
    {'name': 'Omar', 'age': 20, 'country': 'Italie', 'date': "2023-10-26"}
]

df = spark.createDataFrame(d).withColumn("date", to_date("date"))

window = (Window
          .partitionBy("country", "date")
          .orderBy(desc("age"))
          .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)
df.withColumn("rank", rank().over(window)).show()

+---+---------+----------+-------+----+
|age|  country|      date|   name|rank|
+---+---------+----------+-------+----+
| 50|Allemagne|2023-10-10|Richard|   1|
| 40|   France|2023-10-01|  Alice|   1|
| 28|   France|2023-10-01|   Jane|   2|
| 30|   France|2023-10-10|    Bob|   1|
| 20|   Italie|2023-10-26|   Omar|   1|
+---+---------+----------+-------+----+



## Sort

In [16]:
df = spark.createDataFrame([(2, 20), (3, 30), (1, 10), (1, 30), (1, 20)], ["a", "b"])

assert df.orderBy("a", ascending=False).select("a").first().a == 3
assert df.orderBy(["b", "a"]).first() == Row(1, 10)
assert df.sort(col("a").desc()).first().b == 30
assert df.sort(desc("a")).first().a == 3
assert df.sort("a", desc("b")).first() == Row(1, 30)

## Describe dataframe

In [53]:
df = spark.createDataFrame([(2, 20), (3, 30), (1, 10), (1, 30), (1, 20)], ["a", "b"])

df.describe("a").show()
df.summary("count").show()

+-------+------------------+
|summary|                 a|
+-------+------------------+
|  count|                 5|
|   mean|               1.6|
| stddev|0.8944271909999159|
|    min|                 1|
|    max|                 3|
+-------+------------------+

+-------+---+---+
|summary|  a|  b|
+-------+---+---+
|  count|  5|  5|
+-------+---+---+



## Sample

In [60]:
integers = [1, 2, 2, 3, 4, 4, 5]
dfInt = spark.createDataFrame(integers, IntegerType())

assert [row.value for row in dfInt.sample(True, fraction=0.5, seed=3).collect()] == [4, 5]
assert [row.value for row in dfInt.sample(False, fraction=0.5, seed=3).collect()] == [1, 4, 4]

## Dates

In [16]:
df = spark.createDataFrame([(1, "a"), (2, "b")])
(df
 .withColumn("test", lit(1408024997).cast("timestamp"))
 .withColumn("month", month(col("test")))
 .withColumn("day_of_year", dayofyear(col("test")))
 .withColumn("yo", from_unixtime(lit(1408024997), "EEEE, MMM d, yyyy h:mm a"))
 .withColumn("yo_timestamp", unix_timestamp(lit("02/01/2024 10:48"), "dd/MM/yyyy HH:mm"))
 .show(10, False)
)

+---+---+-------------------+-----+-----------+------------------------------+------------+
|_1 |_2 |test               |month|day_of_year|yo                            |yo_timestamp|
+---+---+-------------------+-----+-----------+------------------------------+------------+
|1  |a  |2014-08-14 16:03:17|8    |226        |Thursday, Aug 14, 2014 4:03 PM|1704188880  |
|2  |b  |2014-08-14 16:03:17|8    |226        |Thursday, Aug 14, 2014 4:03 PM|1704188880  |
+---+---+-------------------+-----+-----------+------------------------------+------------+



In [62]:
dfDates = spark.createDataFrame([("23/01/2022 11:28:12",),("24/01/2022 10:58:34",)], ["date"])
dfDates = (
    dfDates.withColumn("date_timestamp", to_timestamp("date", "dd/MM/yyyy HH:mm:ss"))
    .withColumn("date_date", to_date("date", "dd/MM/yyyy HH:mm:ss"))
)

dfDates.show()
dfDates.printSchema()

+-------------------+-------------------+----------+
|               date|     date_timestamp| date_date|
+-------------------+-------------------+----------+
|23/01/2022 11:28:12|2022-01-23 11:28:12|2022-01-23|
|24/01/2022 10:58:34|2022-01-24 10:58:34|2022-01-24|
+-------------------+-------------------+----------+

root
 |-- date: string (nullable = true)
 |-- date_timestamp: timestamp (nullable = true)
 |-- date_date: date (nullable = true)



## Fill with empty values (na.fill)

In [70]:
df = spark.range(2)
(
    df.withColumn("test_na_1", lit(None).cast(StringType()))
    .withColumn("test_na_2", lit(None).cast(IntegerType()))
    .withColumn("test_na_3", lit(None).cast(IntegerType()))
    .withColumn("test_na_4", lit(None).cast(IntegerType()))
    .na.fill("yo")
    .na.fill({'test_na_2': 30})
    .na.fill(5, "test_na_4")
    .show()
)

+---+---------+---------+---------+---------+
| id|test_na_1|test_na_2|test_na_3|test_na_4|
+---+---------+---------+---------+---------+
|  0|       yo|       30|     null|        5|
|  1|       yo|       30|     null|        5|
+---+---------+---------+---------+---------+



## Joins

In [73]:
a = spark.createDataFrame([("Alice", 1), ("Bob", 2), ("Rachid", 3)], "name:string, id: int")
b = spark.createDataFrame([("Franck", 3, 42), ("Bernard", 4, 42), ("Ramzy", 5, 43)], "name:string, id: int, dept_id: int")

a.crossJoin(b).show()

a.join(b, a.id == b.id).show()
a.join(b, [a.id == b.id])
a.join(b, ["id", "name"])
a.alias("a").join(b.alias("b"), [col("a.id") == col("b.id"), col("a.name") == col("b.name")])
a.join(b.select("dept_id"), col("id") == col("dept_id"))

a.join(b, ["id", "name"], "outer").show()

+------+---+-------+---+-------+
|  name| id|   name| id|dept_id|
+------+---+-------+---+-------+
| Alice|  1| Franck|  3|     42|
| Alice|  1|Bernard|  4|     42|
| Alice|  1|  Ramzy|  5|     43|
|   Bob|  2| Franck|  3|     42|
|   Bob|  2|Bernard|  4|     42|
|   Bob|  2|  Ramzy|  5|     43|
|Rachid|  3| Franck|  3|     42|
|Rachid|  3|Bernard|  4|     42|
|Rachid|  3|  Ramzy|  5|     43|
+------+---+-------+---+-------+

+------+---+------+---+-------+
|  name| id|  name| id|dept_id|
+------+---+------+---+-------+
|Rachid|  3|Franck|  3|     42|
+------+---+------+---+-------+

+---+-------+-------+
| id|   name|dept_id|
+---+-------+-------+
|  1|  Alice|   null|
|  2|    Bob|   null|
|  3| Franck|     42|
|  3| Rachid|   null|
|  4|Bernard|     42|
|  5|  Ramzy|     43|
+---+-------+-------+



In [133]:
small_df = spark.createDataFrame([1, 2, 3], IntegerType())
large_df = spark.createDataFrame([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], IntegerType())

large_df.join(broadcast(small_df), "v").show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
+-----+



## Union

In [18]:
a = spark.createDataFrame([("Alice", 1), ("Bob", 2), ("Rachid", 3)], "name:string, id: int")
b = spark.createDataFrame([("Franck", 30), ("Bernard", 40), ("Ramzy", 50)], "name:string, age: int")

a.union(b).show()
a.unionByName(b, allowMissingColumns=True).show()

+-------+---+
|   name| id|
+-------+---+
|  Alice|  1|
|    Bob|  2|
| Rachid|  3|
| Franck| 30|
|Bernard| 40|
|  Ramzy| 50|
+-------+---+

+-------+----+----+
|   name|  id| age|
+-------+----+----+
|  Alice|   1|null|
|    Bob|   2|null|
| Rachid|   3|null|
| Franck|null|  30|
|Bernard|null|  40|
|  Ramzy|null|  50|
+-------+----+----+

+-------+
|   name|
+-------+
|  Alice|
|    Bob|
| Rachid|
| Franck|
|Bernard|
|  Ramzy|
+-------+



## Cast

In [25]:
df.withColumn("test", col("_1").cast(StringType())).printSchema()
df.withColumn("test", col("_1").cast("string")).printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- test: string (nullable = true)

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- test: string (nullable = true)



## UDFs

In [80]:
df = spark.createDataFrame([(1,), (None,)])

def add42(n):
    if n is None:
        return
    return n + 42
    
df.createOrReplaceTempView("test")

spark.udf.register("ADD_42", add42)

spark.sql("select _1, ADD_42(_1) from test").show()

24/01/23 22:38:23 WARN SimpleFunctionRegistry: The function add_42 replaced a previously registered function.
+----+----------+
|  _1|ADD_42(_1)|
+----+----------+
|   1|        43|
|null|      null|
+----+----------+



In [81]:
df = spark.createDataFrame([(1,), (None,)])

def multiply_by_10(n):
    if n is None:
        return
    return n * 10

multiply_by_10_UDF = udf(multiply_by_10, IntegerType())

df.withColumn("test", multiply_by_10_UDF("_1")).show()

+----+----+
|  _1|test|
+----+----+
|   1|  10|
|null|null|
+----+----+



## Cache and Persist

In [11]:
from pyspark import StorageLevel

# cache in MEMORY_AND_DISK_DESER (default)
df.cache().count()
assert str(df.storageLevel).startswith("Disk") == True
print(df.storageLevel)

df.unpersist()

# cache in MEMORY_ONLY
df.persist(StorageLevel.MEMORY_ONLY)
assert str(df.storageLevel).startswith("Memory") == True
print(df.storageLevel)

# stores dataframe on two different executors, utilizing the executors' memory as much as possible, but not writing anything to disk.
df.persist(StorageLevel.MEMORY_ONLY_2).count()

df.unpersist()

assert df.is_cached == False

Disk Memory Deserialized 1x Replicated
Memory Serialized 1x Replicated
23/12/22 22:11:11 WARN CacheManager: Asked to cache already cached data.


## Write dataframe

In [82]:
(
    df
    .write
    .partitionBy("_1")
    .mode("overwrite")
    .parquet("/home/krebai/tmp")
)

AnalysisException: Cannot use all columns for partition columns

In [160]:
df.write.mode("overwrite").json("/home/krebai/tmp/json")

schema = StructType([
    StructField("_1", LongType(), True),
    StructField("_2", StringType(), True)
])

spark.read.json("/home/krebai/tmp/json", schema=schema).show()

+---+---+
| _1| _2|
+---+---+
|  1|  a|
|  2|  b|
+---+---+



## Accumulators

In [11]:
df = spark.createDataFrame([('yo Inc.',), ('test',), ('Inc. fada',)], 'name: string')

accum = sc.accumulator(0)

def check(row):
    if 'Inc.' in row['name']:
        accum.add(1)

df.foreach(check)
print(accum.value)
        

2


## Compute number of business days between 2 dates

In [87]:
df_facts = spark.createDataFrame(
    [('data1', '2023-12-18', '2023-12-24'),
     ('data1', '2022-05-08', '2022-05-21')],
    ['data', 'start_date', 'end_date']
)

In [99]:
(
    df_facts.withColumn("test", sequence(to_date("start_date"), to_date("end_date")))
    .withColumn("business_days_1", expr("size(filter(test, x -> dayofweek(x) != 1 and dayofweek(x) != 7))"))
    .withColumn("business_days_2", size(filter("test", lambda x: (dayofweek(x) != 1) & (dayofweek(x) != 7))))
    .drop("test")
    .show()
)

+-----+----------+----------+---------------+---------------+
| data|start_date|  end_date|business_days_1|business_days_2|
+-----+----------+----------+---------------+---------------+
|data1|2023-12-18|2023-12-24|              5|              5|
|data1|2022-05-08|2022-05-21|             10|             10|
+-----+----------+----------+---------------+---------------+

